# Notebook: Run NN on KG embeddings
<b>Author</b>: Ian Coleman <br/>
<b>Function</b>: Take word2vec embeddings of walking-rdf-owl random walks from my original Knowledge Graph

In [ ]:
import pandas as pd
import numpy as np
import subprocess

In [ ]:
# Output format:
id_of_gene_upreg go_function_uri


In [ ]:
# Input:
c-g those with dir evid 